In [12]:
from huggingface_hub import notebook_login
notebook_login()

In [13]:
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=token)
model_name = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=token)

NameError: name 'LlamaTokenizer' is not defined

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM

#model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)#trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    torch_dtype=torch.float16, 
    trust_remote_code=True
)

tokenizer.pad_token = tokenizer.eos_token


NameError: name 'model_name' is not defined

In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import os # Import the os module

def parse_semeval_xml(file_path):
    """
    Parses SemEval 2016 Task 5 XML file to extract sentences, aspect terms, and polarities.

    Args:
        file_path (str): Path to the XML file.

    Returns:
        list: A list of dictionaries, each containing 'sentence', 'aspect_term',
              'polarity', and 'sentence_id'. Returns empty list if file not found.
    """
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}")
        return []

    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []

    for review in root.findall('Review'):
        for sentences_node in review.findall('sentences'):
            for sentence_node in sentences_node.findall('sentence'):
                sentence_id = sentence_node.get('id')
                text = sentence_node.find('text').text

                opinions = sentence_node.find('Opinions')
                if opinions is not None:
                    for opinion in opinions.findall('Opinion'):
                        aspect_term = opinion.get('target')
                        polarity = opinion.get('polarity')
                        # We only care about explicit aspects with valid polarity
                        if aspect_term != 'NULL' and polarity in ['positive', 'negative', 'neutral']:
                             data.append({
                                 'sentence_id': sentence_id,
                                 'sentence': text,
                                 'aspect_term': aspect_term,
                                 'polarity': polarity
                             })
    return data

# --- Configuration ---
train_file = 'Laptops_Train_sb1.xml' # e.g., './data/ABSA16_Restaurants_Train.xml'
test_file = 'laptops_test_sb1.xml'   # e.g., './data/ABSA16_Restaurants_Test.xml'

# --- Load Data ---
print("Loading training data...")
train_data = parse_semeval_xml(train_file)
print("Loading test data...")
test_data = parse_semeval_xml(test_file)

if not train_data or not test_data:
    print("\nError loading data. Please check file paths and ensure files exist.")
    # Exit or handle error appropriately in a full script
else:
    print(f"\nLoaded {len(train_data)} training samples.")
    print(f"Loaded {len(test_data)} test samples.")

    # Convert to Pandas DataFrame for easier handling (optional)
    train_df = pd.DataFrame(train_data)
    test_df = pd.DataFrame(test_data)

    print("\nSample Training Data:")
    print(train_df.head())

    print("\nSample Test Data:")
    print(test_df.head())

    # Check distribution of polarities
    print("\nTest Set Polarity Distribution:")
    print(test_df['polarity'].value_counts())


Loading training data...
Loading test data...

Loaded 2909 training samples.
Loaded 801 test samples.

Sample Training Data:
  sentence_id                                           sentence aspect_term  \
0        79:1             This computer is absolutely AMAZING!!!        None   
1        79:2                        10 plus hours of battery...        None   
2        79:3  super fast processor and really nice graphics ...        None   
3        79:3  super fast processor and really nice graphics ...        None   
4        79:4  and plenty of storage with 250 gb(though I wil...        None   

   polarity  
0  positive  
1  positive  
2  positive  
3  positive  
4  positive  

Sample Test Data:
                  sentence_id  \
0  B0074703CM_108_ANONYMOUS:0   
1  B0074703CM_108_ANONYMOUS:1   
2  B0074703CM_108_ANONYMOUS:4   
3  B0074703CM_108_ANONYMOUS:4   
4   B00GJUQ4Z0_10_ANONYMOUS:0   

                                            sentence aspect_term  polarity  
0  Well, my fir

In [9]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-2-7b-hf")

c:\Users\dell\anaconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like meta-llama/Llama-2-7b-hf is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# --- Configuration ---
# This directory should contain files like 'config.json', 'pytorch_model.bin.index.json',
# tokenizer files ('tokenizer.model', 'tokenizer_config.json' etc.)
model_path = "D:\Python Scripts\LLMs\Llama-2-7b-hf"
# Using r"..." (raw string) helps with backslashes on Windows # Or llama-2-13b-hf, etc.

# --- Load Model and Tokenizer ---
print(f"Loading model and tokenizer from: {model_path}")

try:
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    # Add padding token if missing (common for Llama)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token # Use EOS token as PAD token

    # Load model
    # Use device_map='auto' to automatically distribute across available GPUs
    # Use load_in_8bit=True or load_in_4bit=True for quantization (requires bitsandbytes)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        trust_remote_code=True,
        torch_dtype=torch.float16, # Use float16 for faster inference & less memory
        device_map='auto',         # Automatically use available GPUs
        # load_in_8bit=True,       # Uncomment for 8-bit quantization
        # load_in_4bit=True,       # Uncomment for 4-bit quantization (NF4)
    )

    # Set model to evaluation mode
    model.eval()

    print("Model and tokenizer loaded successfully.")

    # --- Create a Hugging Face pipeline for easier text generation ---
    # Note: Adjust max_new_tokens based on expected output length
    # Temperature=0.1 encourages more deterministic output
    text_generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
        max_new_tokens=10, # Only need a short output (positive/negative/neutral)
        temperature=0.1,
        do_sample=True # Sample slightly even with low temp
    )
    print("Text generation pipeline created.")

except OSError as e:
    print(f"Error loading model/tokenizer: {e}")
    print(f"Please ensure the path '{model_path}' is correct and contains the necessary model files.")
    # Handle error appropriately
except Exception as e:
    print(f"An unexpected error occurred during model loading: {e}")
    # Handle error appropriately



Loading model and tokenizer from: D:\Python Scripts\LLMs\Llama-2-7b-hf


Using pad_token, but it is not set yet.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and tokenizer loaded successfully.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Text generation pipeline created.


In [16]:
import pandas as pd
import re # For parsing output
from tqdm.auto import tqdm # Progress bar

# --- Prompt Template ---
# This is a crucial part - experiment with different phrasings!
# We constrain the output by explicitly listing the choices.
prompt_template = """Given the following sentence and aspect term, determine the sentiment polarity expressed towards the aspect.

Sentence: "{sentence}"
Aspect Term: "{aspect_term}"

Sentiment (choose one: positive, negative, neutral):"""

def format_prompt(sentence, aspect_term):
    """Formats the input using the prompt template."""
    return prompt_template.format(sentence=sentence, aspect_term=aspect_term)

def parse_prediction(generated_text, original_prompt):
    """
    Parses the model's generated text to extract the sentiment label.
    Handles cases where the model might repeat the prompt or add extra text.
    """
    # Remove the original prompt part from the generated text
    output_only = generated_text[len(original_prompt):].strip().lower()

    # Look for the keywords directly
    if 'positive' in output_only:
        return 'positive'
    elif 'negative' in output_only:
        return 'negative'
    elif 'neutral' in output_only:
        return 'neutral'
    else:
        # Fallback or further parsing logic if needed
        print(f"Warning: Could not parse sentiment from output: '{output_only}'")
        return 'unknown' # Or handle as an error / default

# --- Prediction Loop ---
predictions = []

# Ensure test_df is loaded and is a DataFrame
if 'test_df' in locals() and isinstance(test_df, pd.DataFrame) and 'text_generator' in locals():
    print(f"\nGenerating predictions for {len(test_df)} test samples...")
    # Use tqdm for a progress bar
    for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0], desc="Predicting"):
        sentence = row['sentence']
        aspect_term = row['aspect_term']

        # Create the prompt for the current sample
        prompt = format_prompt(sentence, aspect_term)

        # Generate text using the pipeline
        # Note: Pipelines handle batching internally if you pass a list of prompts,
        # but iterating might be simpler for debugging initially.
        # For larger datasets, consider batching.
        try:
            generated = text_generator(prompt)
            # Extract the generated text part
            generated_text = generated[0]['generated_text']

            # Parse the prediction
            predicted_polarity = parse_prediction(generated_text, prompt)
            predictions.append(predicted_polarity)

        except Exception as e:
            print(f"\nError during prediction for sample {index}: {e}")
            predictions.append('error') # Mark as error

    # Add predictions to the DataFrame
    test_df['predicted_polarity'] = predictions
    print("\nPredictions generated.")
    print(test_df[['sentence', 'aspect_term', 'polarity', 'predicted_polarity']].head(10))
else:
    print("\nSkipping prediction loop: test_df or text_generator not available.")




Generating predictions for 801 test samples...


Predicting:   0%|          | 0/801 [00:00<?, ?it/s]

c:\Users\dell\anaconda3\envs\nlp\lib\site-packages\transformers\pipelines\base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


+1'

<p>'

<p>'

<p>'

Predictions generated.
                                            sentence aspect_term  polarity  \
0  Well, my first apple computer and I am impressed.        None  positive   
1                   Works well, fast and no reboots.        None  positive   
2                                 Glad I did so far.        None  positive   
3                                 Glad I did so far.        None  positive   
4  s.... L .... o..... w....  rea......llllyy  slow.        None  negative   
5                     like seriously  - really slow.        None  negative   
6                                 impossible to use.        None  negative   
7                           cant even read properly.        None  negative   
8                    plus - no russian input ??  wtf        None  negative   
9  What a great laptop, I can run my games and wo...        None  positive   

  predicted_polarity  
0           positive  
1           positive  
2           positive  
3  

In [17]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd

# Ensure test_df exists and has predictions
if 'test_df' in locals() and 'predicted_polarity' in test_df.columns:
    print("\nEvaluating predictions...")

    # Filter out samples where prediction failed or couldn't be parsed
    eval_df = test_df[test_df['predicted_polarity'].isin(['positive', 'negative', 'neutral'])]

    if not eval_df.empty:
        true_labels = eval_df['polarity']
        pred_labels = eval_df['predicted_polarity']

        # Calculate metrics
        accuracy = accuracy_score(true_labels, pred_labels)
        # Use macro F1 for unbalanced classes (common in sentiment analysis)
        macro_f1 = f1_score(true_labels, pred_labels, average='macro')
        # Use weighted F1 as another perspective
        weighted_f1 = f1_score(true_labels, pred_labels, average='weighted')

        print(f"\nEvaluation Results ({len(eval_df)} samples):")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Macro F1-Score: {macro_f1:.4f}")
        print(f"Weighted F1-Score: {weighted_f1:.4f}")

        # Detailed classification report
        print("\nClassification Report:")
        # Define labels to ensure consistent order in the report
        labels = sorted(list(set(true_labels) | set(pred_labels)))
        print(classification_report(true_labels, pred_labels, labels=labels, digits=4))

    else:
        print("No valid predictions available for evaluation.")

    # Analyze errors (optional)
    error_df = test_df[test_df['polarity'] != test_df['predicted_polarity']]
    print(f"\nNumber of prediction errors: {len(error_df)} out of {len(test_df)}")
    if not error_df.empty:
        print("Sample Errors:")
        print(error_df[['sentence', 'aspect_term', 'polarity', 'predicted_polarity']].head())

else:
    print("\nSkipping evaluation: test_df or predictions not available.")




Evaluating predictions...

Evaluation Results (797 samples):
Accuracy: 0.6048
Macro F1-Score: 0.2560
Weighted F1-Score: 0.4588

Classification Report:
              precision    recall  f1-score   support

    negative     0.6667    0.0074    0.0145       272
     neutral     0.0000    0.0000    0.0000        45
    positive     0.6045    1.0000    0.7535       480

    accuracy                         0.6048       797
   macro avg     0.4237    0.3358    0.2560       797
weighted avg     0.5916    0.6048    0.4588       797


Number of prediction errors: 319 out of 801
Sample Errors:
                                            sentence aspect_term  polarity  \
4  s.... L .... o..... w....  rea......llllyy  slow.        None  negative   
5                     like seriously  - really slow.        None  negative   
6                                 impossible to use.        None  negative   
7                           cant even read properly.        None  negative   
8                

c:\Users\dell\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dell\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dell\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
